In [1]:
import pandas as pd
import numpy as np
import os
import random
import pickle as pkl

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable


In [2]:
song_list = os.listdir('../recoded_songs/')

In [3]:
song = pd.read_csv('../recoded_songs/Apex_Blues_A.csv')

In [4]:
song.head()

,state,section
0,E_S,Ia
1,F_L,Ia
2,G_S,Ia
3,Gb_L,Ia
4,E_S,Ia


# Generate index

In [5]:
chord2index = {'PAD':0, 'START':1, 'END':2}
for song in song_list:
    curr_song = pd.read_csv('../recoded_songs/%s'%song)
    chords = curr_song.state.values
    for chord in chords:
        if chord not in chord2index:
            chord2index[chord] = len(chord2index)

In [6]:
len(chord2index)

29

In [23]:
index2chord = {}
for key, val in chord2index.items():
    index2chord[val] = key

# Encode sequences

In [7]:
def encode_progression(s, chord2index, padding_start=True):
    enc = np.array([0,0,1] + [chord2index.get(w) for w in s] + [2])
    return enc
def get_progression(filepath):
    song = pd.read_csv(filepath)
    progression = song.state.values
    return progression

In [8]:
class ProgressionDataset(Dataset):
    def __init__(self, song_paths, chord2index):
        self.paths = song_paths
        self.progressions = []
        for path in self.paths:
            progression = get_progression('../recoded_songs/%s'%path)
            self.progressions.append(encode_progression(progression, chord2index))
            
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        x = self.progressions[idx][:-1]
        y = self.progressions[idx][1:]
        return x, y

In [9]:
train_ds = ProgressionDataset(song_list, chord2index)

In [10]:
batch_size=1
train_dl = DataLoader(train_ds, batch_size=batch_size)

In [11]:
x, y = next(iter(train_dl))

In [12]:
x, y

(tensor([[ 0,  0,  1,  3,  4,  5,  6,  3,  3,  3,  7,  3,  4,  5,  6,  3,  3,  3,
           7,  3,  4,  5,  6,  3,  4,  5,  6,  3,  4,  5,  6,  3,  3,  3,  7,  8,
           9, 10, 11,  8,  8,  8, 12,  8,  9, 10, 11,  8,  8,  8, 12,  3,  4,  5,
           6,  3,  4,  5,  6,  3,  4,  5,  6,  7,  8, 12,  7,  8, 12, 13,  7, 14,
           8,  8, 12, 13,  7, 13,  7,  5, 15, 12, 14, 10, 10, 10, 10, 10, 10, 10,
          14, 10, 10, 10, 10, 10, 14, 16, 15]]),
 tensor([[ 0,  1,  3,  4,  5,  6,  3,  3,  3,  7,  3,  4,  5,  6,  3,  3,  3,  7,
           3,  4,  5,  6,  3,  4,  5,  6,  3,  4,  5,  6,  3,  3,  3,  7,  8,  9,
          10, 11,  8,  8,  8, 12,  8,  9, 10, 11,  8,  8,  8, 12,  3,  4,  5,  6,
           3,  4,  5,  6,  3,  4,  5,  6,  7,  8, 12,  7,  8, 12, 13,  7, 14,  8,
           8, 12, 13,  7, 13,  7,  5, 15, 12, 14, 10, 10, 10, 10, 10, 10, 10, 14,
          10, 10, 10, 10, 10, 14, 16, 15,  2]]))

In [13]:
class MelodyRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MelodyRNN, self).__init__()

        self.embedding = nn.Embedding(input_size, hidden_size, padding_idx=0)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, input_size)
        self.dropout = nn.Dropout(0.3)
        self.hidden = Variable(torch.zeros(1, 1, hidden_size))                
        
    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.dropout(embedded)
        output, self.hidden = self.gru(embedded, self.hidden)
        output = self.out(self.hidden[-1])
        return output, self.hidden

In [14]:
def train_batch(x, y, model, optimizer,
                teacher_forcing_ratio=0.5):

    optimizer.zero_grad()
    
    batch_size = y.size(0)
    target_length = y.size(1)

    output, hidden = model(x)

    loss = 0
    dec_input = y[:,0].unsqueeze(1) # always SOS

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(1, target_length):
        output, hidden = model(dec_input)
        yi =  y[:, di]
        if (yi>0).sum() > 0:
            # ignoring padding
            loss += F.cross_entropy(output, yi, ignore_index = 0, reduction="sum")/(yi>0).sum()
        if use_teacher_forcing:
            dec_input = y[:, di].unsqueeze(1)  # Teacher forcing: Feed the target as the next input
        else:                
            dec_input = output.argmax(dim=1).unsqueeze(1).detach()

    loss.backward(retain_graph=True)

    optimizer.step()

    return loss.item()

def train(model, dec_optimizer, epochs = 10,
          teacher_forcing_ratio=0.5):
    for i in range(epochs):
        total_loss = 0
        total = 0
        model.train()
        for x, y in train_dl:
            x = x.long()
            y = y.long()
            loss = train_batch(x, y, model, dec_optimizer,
                               teacher_forcing_ratio)
            total_loss = loss*x.size(0)
            total += x.size(0)
            
        print("train loss %.3f" % (total_loss / total))

In [15]:
input_size = len(chord2index)
hidden_size = 100
model = MelodyRNN(input_size, hidden_size)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [16]:
train(model, optimizer, epochs = 20)

train loss 17.297
train loss 9.372
train loss 16.253
train loss 8.189
train loss 23.088
train loss 7.817
train loss 7.443
train loss 6.846
train loss 9.054
train loss 13.585
train loss 8.314
train loss 8.633
train loss 6.730
train loss 16.933
train loss 12.293
train loss 8.134
train loss 6.320
train loss 14.737
train loss 7.056
train loss 13.105


# Save model

In [20]:
torch.save(model, 'Full_RNN')

/anaconda3/envs/eventbrite/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MelodyRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [25]:
with open('index2chord.pkl', 'wb') as f:
    pkl.dump(index2chord, f)